In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Required for 3D plotting

# Physical parameters
L = 1.0          # Length of the domain in x and y directions
alpha = 1.0      # Diffusivity constant

# Numerical parameters
Nx = 50          # Number of grid points in x direction
Ny = 50          # Number of grid points in y direction
dx = L / (Nx - 1)
dy = L / (Ny - 1)

# Stability condition for explicit scheme
dt = 0.25 * dx**2 / alpha  # Time step size
Nt = 1000                  # Number of time steps

# Create spatial grids
x = np.linspace(0, L, Nx)
y = np.linspace(0, L, Ny)
X, Y = np.meshgrid(x, y)

# Initialize the solution arrays
u = np.zeros((Nx, Ny))
u_new = np.zeros((Nx, Ny))

# Set initial condition u(x, y, 0) = sin(pi*x/L) * sin(pi*y/L)
for i in range(Nx):
    for j in range(Ny):
        u[i, j] = np.sin(np.pi * x[i] / L) * np.sin(np.pi * y[j] / L)

# Finite difference update function
def update(u, u_new, alpha, dt, dx, dy):
    # Update the interior points
    for i in range(1, Nx - 1):
        for j in range(1, Ny - 1):
            u_new[i, j] = u[i, j] + alpha * dt * (
                (u[i+1, j] - 2*u[i, j] + u[i-1, j]) / dx**2 +
                (u[i, j+1] - 2*u[i, j] + u[i, j-1]) / dy**2
            )
    # Apply boundary conditions (u = 0 on the boundaries)
    u_new[0, :] = 0
    u_new[-1, :] = 0
    u_new[:, 0] = 0
    u_new[:, -1] = 0
    return u_new

# Main time-stepping loop
for n in range(Nt):
    u_new = update(u, u_new, alpha, dt, dx, dy)
    u, u_new = u_new, u  # Swap references

# Analytical solution
def analytical_solution(x, y, t, L, alpha):
    return np.sin(np.pi * x / L) * np.sin(np.pi * y / L) * np.exp(-2 * (np.pi**2) * alpha * t / L**2)

u_analytical = analytical_solution(X, Y, Nt*dt, L, alpha)

# Error calculation
error = np.abs(u - u_analytical)
max_error = np.max(error)
print(f'Maximum error at time t = {Nt*dt:.4f} is {max_error:.6e}')

# Visualization
fig = plt.figure(figsize=(12, 5))

# Numerical solution
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(X, Y, u.T, cmap='viridis')
ax1.set_title('Numerical Solution')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('u(x,y)')

# Analytical solution
ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_surface(X, Y, u_analytical.T, cmap='viridis')
ax2.set_title('Analytical Solution')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('u(x,y)')

plt.tight_layout()
plt.show()

# Error heatmap
plt.figure(figsize=(6, 5))
plt.contourf(X, Y, error.T, 20, cmap='hot')
plt.colorbar(label='Error')
plt.title('Absolute Error Distribution')
plt.xlabel('x')
plt.ylabel('y')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
D_L = 1.0        # Diffusion coefficient
V_H = 1.0        # Molar volume
R = 8.314        # Gas constant (J/(mol*K))
T = 300          # Temperature (K)

# Domain parameters
Lx = 1.0         # Length in x-direction
Ly = 1.0         # Length in y-direction

# Grid parameters
Nx = 50          # Number of grid points in x-direction
Ny = 50          # Number of grid points in y-direction
dx = Lx / (Nx - 1)
dy = Ly / (Ny - 1)

# Time parameters
dt = 0.0001      # Time step size
Nt = 500         # Number of time steps

x = np.linspace(0, Lx, Nx)
y = np.linspace(0, Ly, Ny)
X, Y = np.meshgrid(x, y)

# Define sigma_H
sigma_H = np.sin(np.pi * X / Lx) * np.sin(np.pi * Y / Ly)

# Compute derivatives of sigma_H
sigma_H_x = np.zeros_like(sigma_H)
sigma_H_y = np.zeros_like(sigma_H)
sigma_H_xx = np.zeros_like(sigma_H)
sigma_H_yy = np.zeros_like(sigma_H)

for i in range(1, Nx - 1):
    for j in range(1, Ny - 1):
        sigma_H_x[i, j] = (sigma_H[i+1, j] - sigma_H[i-1, j]) / (2 * dx)
        sigma_H_y[i, j] = (sigma_H[i, j+1] - sigma_H[i, j-1]) / (2 * dy)
        sigma_H_xx[i, j] = (sigma_H[i+1, j] - 2 * sigma_H[i, j] + sigma_H[i-1, j]) / dx**2
        sigma_H_yy[i, j] = (sigma_H[i, j+1] - 2 * sigma_H[i, j] + sigma_H[i, j-1]) / dy**2

# Boundary conditions for sigma_H derivatives
sigma_H_x[0, :] = sigma_H_x[-1, :] = 0
sigma_H_y[:, 0] = sigma_H_y[:, -1] = 0
sigma_H_xx[0, :] = sigma_H_xx[-1, :] = 0
sigma_H_yy[:, 0] = sigma_H_yy[:, -1] = 0

# Initial concentration
C_L = np.exp(-((X - Lx/2)**2 + (Y - Ly/2)**2) / 0.1)

# Prepare to store solutions
C_L_time = [np.copy(C_L)]
C_L_new = np.copy(C_L)

# Update function
def update_C_L(C_L, C_L_new, sigma_H, sigma_H_x, sigma_H_y, sigma_H_xx, sigma_H_yy):
    for i in range(1, Nx - 1):
        for j in range(1, Ny - 1):
            # Spatial derivatives of C_L
            C_L_x = (C_L[i+1, j] - C_L[i-1, j]) / (2 * dx)
            C_L_y = (C_L[i, j+1] - C_L[i, j-1]) / (2 * dy)
            C_L_xx = (C_L[i+1, j] - 2 * C_L[i, j] + C_L[i-1, j]) / dx**2
            C_L_yy = (C_L[i, j+1] - 2 * C_L[i, j] + C_L[i, j-1]) / dy**2
            
            # Additional terms
            term1 = C_L_x * sigma_H_x[i, j] + C_L[i, j] * sigma_H_xx[i, j]
            term2 = C_L_y * sigma_H_y[i, j] + C_L[i, j] * sigma_H_yy[i, j]
            
            RHS = D_L * (C_L_xx + C_L_yy) - (D_L * V_H) / (R * T) * (term1 + term2)
            
            C_L_new[i, j] = C_L[i, j] + dt * RHS
    
    # Apply boundary conditions
    C_L_new[0, :] = 0
    C_L_new[-1, :] = 0
    C_L_new[:, 0] = 0
    C_L_new[:, -1] = 0
    
    return C_L_new

# Time-stepping loop
for n in range(Nt):
    C_L_new = update_C_L(C_L, C_L_new, sigma_H, sigma_H_x, sigma_H_y, sigma_H_xx, sigma_H_yy)
    C_L, C_L_new = C_L_new, C_L
    C_L_time.append(np.copy(C_L))

# Callable function
def get_C_L(x_query, y_query, t_query):
    # Find indices
    i = np.searchsorted(x, x_query) - 1
    j = np.searchsorted(y, y_query) - 1
    i = min(max(i, 0), Nx - 2)
    j = min(max(j, 0), Ny - 2)
    # Interpolation factors
    x1, x2 = x[i], x[i+1]
    y1, y2 = y[j], y[j+1]
    fx = (x_query - x1) / (x2 - x1)
    fy = (y_query - y1) / (y2 - y1)
    # Time index
    t_index = int(t_query / dt)
    t_index = min(max(t_index, 0), Nt)
    # Get C_L at t_index
    C = C_L_time[t_index]
    # Bilinear interpolation
    C_value = (1 - fx) * (1 - fy) * C[i, j] + fx * (1 - fy) * C[i+1, j] + (1 - fx) * fy * C[i, j+1] + fx * fy * C[i+1, j+1]
    return C_value

# Visualization
plt.figure(figsize=(6, 5))
plt.contourf(X, Y, C_L.T, 20, cmap='viridis')
plt.colorbar(label='Concentration')
plt.xlabel('x')
plt.ylabel('y')
plt.title(f'Concentration at time t = {Nt * dt:.4f}')
plt.show()

# Example usage of get_C_L
x_query = 0.5
y_query = 0.5
t_query = Nt * dt
C_value = get_C_L(x_query, y_query, t_query)
print(f'Concentration at (x={x_query}, y={y_query}, t={t_query}) is {C_value}')


In [5]:
import deepxde as dde
import numpy as np
import torch
def pde(x, u):
    du_xx = dde.grad.hessian(u, x)
    return -du_xx - (torch.pi**2) * torch.sin(torch.pi * x)
geom = dde.geometry.Interval(0, 1)
bc = dde.DirichletBC(geom, lambda x: 0, lambda x, on_boundary: on_boundary)
net = dde.nn.FNN([1] + [20]*3 + [1], "tanh", "Glorot normal")
data = dde.data.PDE(geom, pde, bc, num_domain=20, num_boundary=2)
model_strong = dde.Model(data, net)
model_strong.compile("adam", lr=0.001)
losshistory, train_state = model_strong.train(epochs=10000)
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
'compile' took 0.000412 s

Training model...

Step      Train loss              Test loss               Test metric
0         [4.37e+01, 3.08e-02]    [4.37e+01, 3.08e-02]    []  


KeyboardInterrupt: 

In [6]:
import deepxde as dde
import numpy as np
import torch
def weak_form(x,u):
    eq1 = u- torch.sin(torch.pi * x) 
    return eq1
def custom_loss(outputs, inputs):
    integrand = weak_form(outputs, inputs)
    return dde.math.integrate(integrand, inputs)
geom = dde.geometry.Interval(0, 1)
bc = dde.DirichletBC(geom, lambda x: 0, lambda x, on_boundary: on_boundary)
net = dde.nn.FNN([1] + [20]*3 + [1], "tanh", "Glorot normal")
data = dde.data.PDE(geom, weak_form, bc, num_domain=20, num_boundary=2)
model_weak = dde.Model(data, net)
model_weak.compile("adam", lr=0.001)
losshistory, train_state = model_weak.train(epochs=10000)
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
'compile' took 0.000467 s

Training model...

0         [2.81e-01, 6.13e-02]    [2.81e-01, 6.13e-02]    []  
1000      [3.90e-04, 1.92e-05]    [3.90e-04, 1.92e-05]    []  
2000      [8.49e-05, 9.04e-06]    [8.49e-05, 9.04e-06]    []  
3000      [3.69e-05, 1.91e-06]    [3.69e-05, 1.91e-06]    []  
4000      [2.05e-05, 1.01e-06]    [2.05e-05, 1.01e-06]    []  


KeyboardInterrupt: 

In [7]:
def u_exact(x):
    return torch.sin(torch.pi * x)
# Evaluation Points
X = geom.uniform_points(100, True)

# Predictions
y_pred_strong = model_strong.predict(X)
y_pred_weak = model_weak.predict(X)
y_exact = u_exact(torch.tensor(X)).detach().numpy()

# Compute Errors
error_strong = np.mean((y_pred_strong - y_exact)**2)
error_weak = np.mean((y_pred_weak - y_exact)**2)

print(f"Mean Squared Error (Strong Form): {error_strong}")
print(f"Mean Squared Error (Weak Form): {error_weak}")


Mean Squared Error (Strong Form): 0.003856054740026593
Mean Squared Error (Weak Form): 1.623481875867583e-05


In [8]:
import deepxde as dde
import numpy as np
import torch
nu = 0.01 / np.pi
geom = dde.geometry.Interval(0, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)
# Initial condition
def ic_func(x):
    return -np.sin(np.pi * x[:, 0:1])
ic = dde.IC(geomtime, ic_func, lambda _, on_initial: on_initial)
bc = dde.DirichletBC(geomtime, lambda x: 0, lambda x, on_boundary: on_boundary)
net = dde.nn.FNN([2] + [50]*3 + [1], "tanh", "Glorot normal")
def pde(x, u):
    u_t = dde.grad.jacobian(u, x, i=0, j=1)
    u_x = dde.grad.jacobian(u, x, i=0, j=0)
    u_xx = dde.grad.hessian(u, x, component=0, i=0, j=0)
    eq1=u_t + u * u_x - nu * u_xx
    return eq1
data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc, ic],
    num_domain=4000,
    num_boundary=100,
    num_initial=100,
    train_distribution="Sobol",
)

model_strong = dde.Model(data, net)
model_strong.compile("adam", lr=0.001)
losshistory, train_state = model_strong.train(epochs=10000)
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
'compile' took 0.000360 s

Training model...

0         [4.36e-02, 1.15e-02, 4.75e-01][4.36e-02, 1.15e-02, 4.75e-01][]  


C:\Users\yesda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\skopt\sampler\sobol.py:521: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+4001=4001. 
  warnings.warn(
C:\Users\yesda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\skopt\sampler\sobol.py:521: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+101=101. 
  warnings.warn(
C:\Users\yesda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\skopt\sampler\sobol.py:521: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+101=101. 
  warnings.warn(


KeyboardInterrupt: 

In [13]:
import deepxde as dde
import numpy as np
import torch
nu = 0.01 / np.pi
geom = dde.geometry.Interval(0, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)
# Initial condition
def ic_func(x):
    return -np.sin(np.pi * x[:, 0:1])
ic = dde.IC(geomtime, ic_func, lambda _, on_initial: on_initial)
bc = dde.DirichletBC(geomtime, lambda x: 0, lambda x, on_boundary: on_boundary)
net = dde.nn.FNN([2] + [50]*3 + [1], "tanh", "Glorot normal")
def custom_loss(outputs, inputs):
    print(inputs)
    print(outputs)
    # Ensure inputs require gradients
    inputs = inputs.requires_grad_(True)
    x = inputs[:, 0:1]
    t = inputs[:, 1:2]
    u = outputs
    # Test function v (here, we use u as the test function for simplicity)
    v = u
    # Compute gradients
    u_t = dde.grad.jacobian(u, inputs, i=0, j=1)
    u_x = dde.grad.jacobian(u, inputs, i=0, j=0)
    v_x = dde.grad.jacobian(v, inputs, i=0, j=0)
    # Define the integrand
    integrand = (u_t + u * u_x) * v - nu * u_x * v_x
    # Approximate the integral over the domain
    integral = torch.mean(integrand)
    return integral

data = dde.data.TimePDE(geomtime,pde=None,ic_bcs=[ic,bc],num_domain=4000,num_boundary=100,num_initial=100,train_distribution="Sobol",)
model_weak = dde.Model(data, net)
model_weak.compile("adam", lr=0.001, loss=custom_loss)
losshistory, train_state = model_weak.train(epochs=10000)
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
'compile' took 0.000378 s



C:\Users\yesda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\skopt\sampler\sobol.py:521: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+4001=4001. 
  warnings.warn(
C:\Users\yesda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\skopt\sampler\sobol.py:521: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+101=101. 
  warnings.warn(
C:\Users\yesda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\skopt\sampler\sobol.py:521: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+101=101. 
  warnings.warn(


Training model...

tensor([[1.1570],
        [0.9345],
        [0.7874],
        [1.0432],
        [0.6422],
        [1.1170],
        [0.4231],
        [0.6160],
        [1.0420],
        [0.4699],
        [1.1192],
        [0.9314],
        [0.7993],
        [1.1561],
        [0.2153],
        [0.3206],
        [1.1412],
        [0.7231],
        [0.9916],
        [1.1429],
        [0.3803],
        [0.9920],
        [0.7048],
        [0.5218],
        [1.0838],
        [0.5575],
        [1.0858],
        [0.8632],
        [0.8700],
        [1.1614],
        [0.1081],
        [0.1619],
        [1.1600],
        [0.8354],
        [0.8983],
        [1.1037],
        [0.5140],
        [1.0639],
        [0.5695],
        [0.7469],
        [0.9641],
        [0.3350],
        [1.1512],
        [1.0185],
        [0.6832],
        [1.1303],
        [0.3722],
        [0.2683],
        [1.1498],
        [0.7618],
        [0.9626],
        [1.1323],
        [0.4253],
        [1.0180],
        [

ValueError: j=1 is not valid.

In [ ]:
def u_exact(x):
    return -np.exp(-np.pi**2 * nu * x[:, 1:2]) * np.sin(np.pi * x[:, 0:1])
X = geomtime.random_points(1000)
y_pred_strong = model_strong.predict(X)
y_pred_weak = model_weak.predict(X)
y_exact = u_exact(X)
error_strong = np.mean((y_pred_strong - y_exact) ** 2)
error_weak = np.mean((y_pred_weak - y_exact) ** 2)
print(f"Mean Squared Error (Strong Form): {error_strong}")
print(f"Mean Squared Error (Weak Form): {error_weak}")


In [ ]:
import deepxde as dde
import numpy as np
import torch

# Viscosity
nu = 0.01

# Geometry for space-time domain
geom = dde.geometry.Rectangle([0, 0], [1, 1])
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)


# Define Dirichlet boundary conditions (no-slip boundary condition)
bc_u = dde.DirichletBC(geomtime, lambda x: 0, lambda x, on_boundary: on_boundary, component=0)
bc_v = dde.DirichletBC(geomtime, lambda x: 0, lambda x, on_boundary: on_boundary, component=1)

# Strong form of the Navier-Stokes equations
def navier_stokes_pde(x, u):
    # Separate the components of the output
    u_comp, v_comp, p_comp = u[:, 0:1], u[:, 1:2], u[:, 2:3]

    # Time derivatives
    u_t = dde.grad.jacobian(u_comp, x, i=0, j=2)
    v_t = dde.grad.jacobian(v_comp, x, i=0, j=2)
    
    # Spatial derivatives for u
    u_x = dde.grad.jacobian(u_comp, x, i=0, j=0)
    u_y = dde.grad.jacobian(u_comp, x, i=0, j=1)
    u_xx = dde.grad.hessian(u_comp, x, component=0, i=0, j=0)
    u_yy = dde.grad.hessian(u_comp, x, component=0, i=1, j=1)

    # Spatial derivatives for v
    v_x = dde.grad.jacobian(v_comp, x, i=0, j=0)
    v_y = dde.grad.jacobian(v_comp, x, i=0, j=1)
    v_xx = dde.grad.hessian(v_comp, x, component=0, i=0, j=0)
    v_yy = dde.grad.hessian(v_comp, x, component=0, i=1, j=1)

    # Spatial derivatives for p
    p_x = dde.grad.jacobian(p_comp, x, i=0, j=0)
    p_y = dde.grad.jacobian(p_comp, x, i=0, j=1)

    # Define the residuals for the momentum equations
    momentum_u = u_t + (u_comp * u_x + v_comp * u_y) + p_x - nu * (u_xx + u_yy)
    momentum_v = v_t + (u_comp * v_x + v_comp * v_y) + p_y - nu * (v_xx + v_yy)
    
    # Define the residual for incompressibility
    continuity = u_x + v_y
    
    return [momentum_u, momentum_v, continuity]

# Define the neural network architecture
net = dde.nn.FNN([3] + [50] * 4 + [3], "tanh", "Glorot normal")

# Data object with strong form PDE definition
data = dde.data.TimePDE(
    geomtime,
    pde=navier_stokes_pde,
    ic_bcs=[bc_u, bc_v],
    num_domain=4000,
    num_boundary=400,
    num_initial=400,
    train_distribution="Sobol",
)

# Model and compilation
model = dde.Model(data, net)
model.compile("adam", lr=0.001)

# Train the model
losshistory, train_state = model.train(epochs=10000)

# Sample points for evaluation
X = geomtime.random_points(1000)
y_pred = model.predict(X)

# Separate predictions for u, v, and p
u_pred = y_pred[:, 0]
v_pred = y_pred[:, 1]
p_pred = y_pred[:, 2]

# Print or visualize predictions
print("Predicted velocity (u):", u_pred[:10])
print("Predicted velocity (v):", v_pred[:10])
print("Predicted pressure (p):", p_pred[:10])


In [ ]:
import deepxde as dde
import numpy as np
import torch

# Set viscosity
nu = 0.01

# Define the computational domain (2D space + time)
geom = dde.geometry.Rectangle([0, 0], [1, 1])
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# Define Dirichlet boundary conditions (no-slip conditions on the walls)
bc_u = dde.DirichletBC(geomtime, lambda x: 0, lambda x, on_boundary: on_boundary, component=0)
bc_v = dde.DirichletBC(geomtime, lambda x: 0, lambda x, on_boundary: on_boundary, component=1)

# Custom weak form loss function for the Navier-Stokes equations
def custom_loss(outputs, inputs):
    # Ensure inputs require gradients for autograd
    inputs = inputs.clone().detach().requires_grad_(True)
    x=inputs[:, 0:1]
    y=inputs[:, 1:2]
    t=inputs[:, 2:3]
    u= outputs[:, 0:1] 
    v= outputs[:, 1:2]
    p= outputs[:, 2:3]

    # Test functions (here, we use u, v, p as test functions for simplicity)
    w_u, w_v, w_p = u, v, p

    # Compute gradients for weak form expressions
    u_t = dde.grad.jacobian(u, inputs, i=0, j=2)
    v_t = dde.grad.jacobian(v, inputs, i=0, j=2)
    u_x = dde.grad.jacobian(u, inputs, i=0, j=0)
    u_y = dde.grad.jacobian(u, inputs, i=0, j=1)
    v_x = dde.grad.jacobian(v, inputs, i=0, j=0)
    v_y = dde.grad.jacobian(v, inputs, i=0, j=1)
    p_x = dde.grad.jacobian(p, inputs, i=0, j=0)
    p_y = dde.grad.jacobian(p, inputs, i=0, j=1)
    
    # Define the weak form integrands for each equation
    integrand_u = u_t * w_u + (u * u_x + v * u_y) * w_u + p_x * w_u - nu * (u_x**2 + u_y**2) * w_u
    integrand_v = v_t * w_v + (u * v_x + v * v_y) * w_v + p_y * w_v - nu * (v_x**2 + v_y**2) * w_v
    integrand_p = (u_x + v_y) * w_p
    
    # Approximate integral as mean of integrands (numerical integration over collocation points)
    integral_u = torch.mean(integrand_u)
    integral_v = torch.mean(integrand_v)
    integral_p = torch.mean(integrand_p)
    
    # Return the combined weak form residuals for u, v, and p
    return integral_u + integral_v + integral_p

# Define the neural network model architecture
net = dde.nn.FNN([3] + [50] * 4 + [1], "tanh", "Glorot normal")

# Define the data object without specifying a PDE directly (using weak form custom loss)
data = dde.data.TimePDE(
    geomtime,
    pde=None,
    ic_bcs=[bc_u, bc_v],
    num_domain=4000,
    num_boundary=400,
    num_initial=400,
    train_distribution="Sobol",
)

# Define and compile the model with the custom weak form loss function
model = dde.Model(data, net)
model.compile("adam", lr=0.001, loss=custom_loss)

# Train the model
losshistory, train_state = model.train(epochs=10000)

# Sample points for evaluation in the domain
X = geomtime.random_points(1000)
y_pred = model.predict(X)

# Separate predictions for u, v, and p
u_pred, v_pred, p_pred = y_pred[:, 0], y_pred[:, 1], y_pred[:, 2]

# Print predictions or calculate errors if an analytical or numerical solution is available
print("Predicted velocity (u):", u_pred[:10])
print("Predicted velocity (v):", v_pred[:10])
print("Predicted pressure (p):", p_pred[:10])
